In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer

from catboost import Pool, cv,CatBoostRegressor

In [ ]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [ ]:
train_df.shape

In [ ]:
data = train_df[train_df['label'] == 1].drop(['label'], axis=1).dropna()

In [ ]:
data.columns

In [ ]:
cat_features = ['pick_cluster','is_more_than_one_day','pickup_timeslot',
       'day_of_week', 'is_weekday']

features = [
    'additional_fare',
#     'duration', 
#     'meter_waiting',
    'meter_waiting_fare',
    'meter_waiting_till_pickup', 
    'pick_cluster', 
    'is_more_than_one_day', 
    'distance_km',
    'pickup_timeslot',
    'day_of_week', 
    'is_weekday'
]

In [ ]:
X = data[features]
y = data['fare'].values

In [ ]:
train_df_X = train_df[features]
train_df_pool = Pool(data=train_df_X,cat_features=cat_features)
test_df_X = test_df[features]
test_df_pool = Pool(data=test_df_X,cat_features=cat_features)

In [ ]:
train_df_preds = np.zeros(train_df.shape[0])
test_df_preds = np.zeros(test_df.shape[0])

In [ ]:
params = {
    'loss_function':'RMSE',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'RMSE'
}

In [ ]:
kf = KFold(n_splits=3)

In [ ]:
validation_scores = []
models = []
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostRegressor(**params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    validation_score = model.best_score_['validation']['RMSE']
    print('Validation RMSE',validation_score)
    validation_scores.append(validation_score)
    models.append(model)
    
    train_df_preds += model.predict(train_df_pool)
    test_df_preds += model.predict(test_df_pool)

In [ ]:
np.mean(validation_scores), np.std(validation_scores)

In [ ]:
best_model = models[np.argmax(validation_scores)]

In [ ]:
best_model.get_feature_importance(prettified=True)

In [ ]:
train_df['predicted_fare'] = train_df_preds/3
test_df['predicted_fare'] = test_df_preds/3

In [ ]:
train_df['predicted_fare_difference'] = train_df['fare'] - train_df['predicted_fare']
test_df['predicted_fare_difference'] = test_df['fare'] - test_df['predicted_fare']

In [ ]:
train_df['predicted_fare_difference_per_fare'] = train_df['predicted_fare_difference'] / (train_df['fare']+1)
test_df['predicted_fare_difference_per_fare'] = test_df['predicted_fare_difference']  / (test_df['fare']+1)

In [ ]:
sns.distplot(train_df[train_df['label']==1]['predicted_fare_difference'], hist=False)
sns.distplot(train_df[train_df['label']==0]['predicted_fare_difference'], hist=False)

In [ ]:
train_df[train_df['label']==1]['predicted_fare_difference'].describe()

In [ ]:
train_df[train_df['label']==0]['predicted_fare_difference'].describe()

In [ ]:
# train_df = train_df.drop(['predicted_fare'],axis=1)
# test_df = test_df.drop(['predicted_fare'],axis=1)

In [ ]:
train_df.to_csv('train_df_reg.csv',index=False)
test_df.to_csv('test_df_reg.csv',index=False)